# Introduction to Machine Learning
## Project Phase 1 - Spring Semester 1402-03
### Department of Electrical Engineering, Sharif University of Technology
### Instructor: Dr. R.Amiri
### Due Date: Tir 10, 1403 at 23:59

## Machine Unlearning

### Learning Phase
In this section, we will implement the SISA Algorithm and test its performance.

1. **Sharding**: The training data is divided into S smaller subsets, or shards.
2. **Isolation**: Each shard is used to train a separate model or a component of a model independently.
3. **Slicing**: Each shard is further divided into slices, and training occurs in a sequential manner, slice by slice, within each shard.
4. **Aggregation**: The final model is constructed by aggregating the outputs from the independently trained shards.

**Part 1.1:**

For learning phase:

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split, ConcatDataset, Subset
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
import numpy as np
import random

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Define the dataset and transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:12<00:00, 13277446.66it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


در اینجا داده‌ها را طبق الگوریتم سیسا به تعدادی قسمت و لایه تبدیل می‌کنیم.

In [ ]:
# Function to create shards and slices
def create_shards_and_slices(dataset, S, R):
    shards = random_split(dataset, [len(dataset) // S] * S)
    slices = [random_split(shard, [len(shard) // R] * R) for shard in shards]
    return slices

در زیر کد برای این است که بر روی هر شارد و هر لایه‌ آن به ترتیب عمل عمل ترین را انجام دهیم.

داده‌ها را 32تا 32تا لود کرده و الگوریتم ترین را انجام می‌دهیم.

سپس بر طبق معیار گرادیان که آن را برابر صفر قرار می‌دهیم، به مدلمان یاد می‌دهیم.

In [ ]:
# Function to train model on slices
def train_on_slices(S, R, model, slices, epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)
    for i, slice_set in enumerate(slices):
        slice_dataset = ConcatDataset(slice_set)
        dataloader = DataLoader(slice_dataset, batch_size=32, shuffle=True)
        for epoch in range(epochs[i]):
            model.train()
            for images, labels in dataloader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        torch.save(model.state_dict(), f'model_slice_{S}_{R}_{i}.pth')
    return model

در اینجا هم کدی برای گرفتن خروجی‌های مد نظر زده‌ایم.

بر روی تمامی داده‌ها فرق بین خروجی مدل ما و لیبل وافعی را ثبت کرده و با توجه به آنها خروجی‌ها را به دست می‌آوریم.(با کمک توابع پیش فرض)

In [ ]:
# Define evaluation metrics
def evaluate_model(model, test_loader):
    model.eval()
    y_true, y_pred = [], []
    y_score = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            y_score.extend(outputs.cpu().numpy())
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    y_true_one_hot = torch.nn.functional.one_hot(torch.tensor(y_true), num_classes=10).numpy()
    auroc = roc_auc_score(y_true_one_hot, y_score, multi_class='ovr')
    return f1, accuracy, precision, recall, auroc

از اینجا به بعد برای هر کدام از حالات

S
و
R

برنامه را ران می‌کنیم.

In [ ]:
# Load dataset and create DataLoader
# 5 5
S = 5
R = 5
slices = create_shards_and_slices(dataset, S=5, R=5)
model1 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model1.fc = nn.Linear(model1.fc.in_features, 10)  # Adjusting for CIFAR-10
model1 = train_on_slices(S, R, model1, slices, epochs=[10, 10, 10, 10, 10])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
f1, accuracy, precision, recall, auroc = evaluate_model(model1, test_loader)
print(f'F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 205MB/s]


F1 Score: 0.8569020180441432, Accuracy: 0.8566, Precision: 0.8612518720489163, Recall: 0.8566, AUROC: 0.9839586666666668


In [ ]:
# 5 10
S,R = 5,10
slices = create_shards_and_slices(dataset, S=5, R=10)
model2 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model2.fc = nn.Linear(model2.fc.in_features, 10)  # Adjusting for CIFAR-10
model2 = train_on_slices(S, R, model2, slices, epochs=[10, 10, 10, 10, 10])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
f1, accuracy, precision, recall, auroc = evaluate_model(model2, test_loader)
print(f'F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

F1 Score: 0.8662552022576003, Accuracy: 0.8666, Precision: 0.8698110303826299, Recall: 0.8666, AUROC: 0.9862109222222222


In [ ]:
# 5 20
S,R = 5,20
slices = create_shards_and_slices(dataset, S=5, R=20)
model3 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model3.fc = nn.Linear(model3.fc.in_features, 10)  # Adjusting for CIFAR-10
model3 = train_on_slices(S, R, model3, slices, epochs=[10, 10, 10, 10, 10])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
f1, accuracy, precision, recall, auroc = evaluate_model(model3, test_loader)
print(f'F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

F1 Score: 0.8557526062540055, Accuracy: 0.8554, Precision: 0.8591821335757857, Recall: 0.8554, AUROC: 0.9833812222222222


In [ ]:
# 10 5
S,R = 10,5
slices = create_shards_and_slices(dataset, S=10, R=5)
model4 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model4.fc = nn.Linear(model4.fc.in_features, 10)  # Adjusting for CIFAR-10
model4 = train_on_slices(S, R, model4, slices, epochs=[10, 10, 10, 10, 10, 10, 10, 10, 10, 10])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
f1, accuracy, precision, recall, auroc = evaluate_model(model4, test_loader)
print(f'F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

5
F1 Score: 0.8480197688269827, Accuracy: 0.848, Precision: 0.849958646261961, Recall: 0.848, AUROC: 0.980687811111111


In [ ]:
# 10 10
S,R = 10,10
slices = create_shards_and_slices(dataset, S=10, R=10)
model5 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model5.fc = nn.Linear(model5.fc.in_features, 10)  # Adjusting for CIFAR-10
model5 = train_on_slices(S, R, model5, slices, epochs=[10, 10, 10, 10, 10, 10, 10, 10, 10, 10])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
f1, accuracy, precision, recall, auroc = evaluate_model(model5, test_loader)
print(f'F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 72.5MB/s]


F1 Score: 0.8353651532954307, Accuracy: 0.8361, Precision: 0.8373842597494878, Recall: 0.8361, AUROC: 0.9803153388888889


In [ ]:
# 10 20
S,R = 10,20
slices = create_shards_and_slices(dataset, S=10, R=20)
model6 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model6.fc = nn.Linear(model6.fc.in_features, 10)  # Adjusting for CIFAR-10
model6 = train_on_slices(S, R, model6, slices, epochs=[10, 10, 10, 10, 10, 10, 10, 10, 10, 10])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
f1, accuracy, precision, recall, auroc = evaluate_model(model6, test_loader)
print(f'F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

In [ ]:
# 20 5
S,R = 20,5
slices = create_shards_and_slices(dataset, S=20, R=5)
model7 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model7.fc = nn.Linear(model7.fc.in_features, 10)  # Adjusting for CIFAR-10
model7 = train_on_slices(S, R, model7, slices, epochs=[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
f1, accuracy, precision, recall, auroc = evaluate_model(model7, test_loader)
print(f'F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

In [ ]:
# 20 10
S,R = 20,10
slices = create_shards_and_slices(dataset, S=20, R=10)
model8 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model8.fc = nn.Linear(model8.fc.in_features, 10)  # Adjusting for CIFAR-10
model8 = train_on_slices(S, R, model8, slices, epochs=[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
f1, accuracy, precision, recall, auroc = evaluate_model(model8, test_loader)
print(f'F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 139MB/s]


In [ ]:
# 20 20
S,R = 20,20
slices = create_shards_and_slices(dataset, S=20, R=20)
model9 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model9.fc = nn.Linear(model9.fc.in_features, 10)  # Adjusting for CIFAR-10
model9 = train_on_slices(S, R, model9, slices, epochs=[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
f1, accuracy, precision, recall, auroc = evaluate_model(model9, test_loader)
print(f'F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

**Part1.2:**

For unlearning phase.

کدی که زدیم، تا حدی شبیه کد برای یادگیری اولیه است.

با این تفاوت که اگر فرقی در داده‌های آن اسلایس نبود، مدل ما ری‌اکشنی نشان نداده و ادامه می‌دهد.

In [ ]:
# Unlearning Phase
# In this phase, we will implement the unlearning algorithm on all trained models and remove 500 randomly chosen data points.

# Function to unlearn data
def unlearn_data(S, R, model, data_to_forget, slices, epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)
    for i, slice_set in enumerate(slices):
        updated_slice = ConcatDataset([data for data in slice_set if data not in data_to_forget])
        dataloader = DataLoader(updated_slice, batch_size=32, shuffle=True)
        for epoch in range(epochs[i]):
            model.train()
            for images, labels in dataloader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        torch.save(model.state_dict(), f'unlearned_model_slice_{S}_{R}_{i}.pth')
    return model
# Select 500 data points to forget
data_to_forget, other_dataset = random_split(dataset, [500, len(dataset) - 500])

In [ ]:
# 5 5
S,R = 5,5
model11 = unlearn_data(S, R, model1, data_to_forget, slices, epochs=[10, 10, 10, 10, 10])
f1, accuracy, precision, recall, auroc = evaluate_model(model1, test_loader)
print(f'Unlearned Model - F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

Unlearned Model - F1 Score: 0.8666689852835648, Accuracy: 0.8647, Precision: 0.8753282073280295, Recall: 0.8647, AUROC: 0.9871240666666667


In [ ]:
# 5 10
S,R = 5,10
model21 = unlearn_data(S, R, model2, data_to_forget, slices, epochs=[10, 10, 10, 10, 10])
f1, accuracy, precision, recall, auroc = evaluate_model(model2, test_loader)
print(f'Unlearned Model - F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

Unlearned Model - F1 Score: 0.8717827619842392, Accuracy: 0.87, Precision: 0.883633839706258, Recall: 0.87, AUROC: 0.987616


In [ ]:
# 5 20
S,R = 5,20
model31 = unlearn_data(S, R, model3, data_to_forget, slices, epochs=[10, 10, 10, 10, 10])
f1, accuracy, precision, recall, auroc = evaluate_model(model3, test_loader)
print(f'Unlearned Model - F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

Unlearned Model - F1 Score: 0.8797749882376206, Accuracy: 0.8806, Precision: 0.8802754874917887, Recall: 0.8806, AUROC: 0.9879218444444444


In [ ]:
# 10 5
S,R = 10,5
model41 = unlearn_data(S, R, model4, data_to_forget, slices, epochs=[10, 10, 10, 10, 10, 10, 10, 10, 10, 10])
f1, accuracy, precision, recall, auroc = evaluate_model(model4, test_loader)
print(f'Unlearned Model - F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

Unlearned Model - F1 Score: 0.8585644084380872, Accuracy: 0.8577, Precision: 0.8682500900764355, Recall: 0.8577, AUROC: 0.9853614111111112


In [ ]:
# 10 10
S,R = 10,10
model51 = unlearn_data(S, R, model5, data_to_forget, slices, epochs=[10, 10, 10, 10, 10, 10, 10, 10, 10, 10])
f1, accuracy, precision, recall, auroc = evaluate_model(model5, test_loader)
print(f'Unlearned Model - F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

In [ ]:
# 10 20
S,R = 10,20
model61 = unlearn_data(S, R, model6, data_to_forget, slices, epochs=[10, 10, 10, 10, 10, 10, 10, 10, 10, 10])
f1, accuracy, precision, recall, auroc = evaluate_model(mode6, test_loader)
print(f'Unlearned Model - F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

In [ ]:
# 20 5
S,R = 20,5
model71 = unlearn_data(S, R, model7, data_to_forget, slices, epochs=[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10])
f1, accuracy, precision, recall, auroc = evaluate_model(model7, test_loader)
print(f'Unlearned Model - F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

In [ ]:
# 20 10
S,R = 20,10
model81 = unlearn_data(S, R, model8, data_to_forget, slices, epochs=[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10])
f1, accuracy, precision, recall, auroc = evaluate_model(model8, test_loader)
print(f'Unlearned Model - F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

In [ ]:
# 20 20
S,R = 20,20
model91 = unlearn_data(S, R, model9, data_to_forget, slices, epochs=[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10])
f1, accuracy, precision, recall, auroc = evaluate_model(mode9, test_loader)
print(f'Unlearned Model - F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

**Part 1.3:**

For example we use train model and untrain model of S=5 & R=5.

در اینجا توابعی که نیاز بوده را آورده‌ایم.

مانند

1. compute loss
2. train attacker model
3. ...

که همه این توابع در بخش بعد هم توضیح داده شده‌اند.

In [ ]:
# Membership Inference Attack
def membership_inference_attack(losses_train, losses_test):
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import cross_val_score
    X = losses_train + losses_test
    y = [1] * len(losses_train) + [0] * len(losses_test)
    clf = LogisticRegression(random_state=0).fit(X, y)
    return clf

def compute_losses(model, dataset):
    model.eval()
    losses = []
    criterion = nn.CrossEntropyLoss()
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            losses.append(loss.item())
    return losses

def train_attacker_model(losses_train, losses_test):
    from sklearn.linear_model import LogisticRegression
    X = losses_train + losses_test
    y = [1] * len(losses_train) + [0] * len(losses_test)
    clf = LogisticRegression(random_state=0).fit([[x] for x in X], y)
    return clf

def cross_val_score(clf, X, y, cv=5):
    from sklearn.model_selection import StratifiedKFold
    skf = StratifiedKFold(n_splits=cv)
    scores = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = [X[i] for i in train_index], [X[i] for i in test_index]
        y_train, y_test = [y[i] for i in train_index], [y[i] for i in test_index]
        clf.fit([[x] for x in X_train], y_train)
        score = clf.score([[x] for x in X_test], y_test)
        scores.append(score)
    return sum(scores) / len(scores)

دیتاست 1: بخشی از دیتاست ترین با وجود داده‌های فراموشی. کل داده‌ها را در نظر گرفتیم.

دیتاست 2: یک بخشی از دیتاست تست. test_subset1

دیتاست 3: بخشی از دیتاست ترین بدون وجود داده‌های فراموشی. کل داده‌ها به جز داده‌های فراموشی.

دیتاست 4: بخش دیگری از دیتاست تست. test_subset2

In [ ]:
split_ratio = 0.5  # 50% split
split_size = int(len(test_dataset) * split_ratio)
remaining_size = len(test_dataset) - split_size

# Split the test_dataset into two subsets
test_subset1, test_subset2 = random_split(test_dataset, [split_size, remaining_size])

1)	لاس‌های دیتاست 1 و 2 در مدل ترین شده و قبل از unlearn.

2)	لاس‌های دیتاست 3 و 4 در مدل ترین شده و قبل از unlearn.

In [ ]:
baseline_losses_train_1 = compute_losses(model1, dataset)
baseline_losses_test_1 = compute_losses(model1, test_subset1)
baseline_losses_train_2 = compute_losses(model1, other_dataset)
baseline_losses_test_2 = compute_losses(model1, test_subset2)

# Combine and cross-validate for baseline model
clf_baseline_1 = train_attacker_model(baseline_losses_train_1, baseline_losses_test_1)
clf_baseline_2 = train_attacker_model(baseline_losses_train_2, baseline_losses_test_2)

X_baseline_1 = baseline_losses_train_1 + baseline_losses_test_1
y_baseline_1 = [1] * len(baseline_losses_train_1) + [0] * len(baseline_losses_test_1)
cv_score_baseline_1 = cross_val_score(clf_baseline_1, X_baseline_1, y_baseline_1)

X_baseline_2 = baseline_losses_train_2 + baseline_losses_test_2
y_baseline_2 = [1] * len(baseline_losses_train_2) + [0] * len(baseline_losses_test_2)
cv_score_baseline_2 = cross_val_score(clf_baseline_2, X_baseline_2, y_baseline_2)

# Print the cross-validation scores
print("Cross-validation score for baseline model (subset 1):", cv_score_baseline_1)
print("Cross-validation score for baseline model (subset 2):", cv_score_baseline_2)

Cross-validation score for baseline model (subset 1): 0.9143229256511989
Cross-validation score for baseline model (subset 2): 0.907563025210084


3)	لاس‌های دیتاست 1 و 2 در مدل بعد از unlearn.

4)	لاس‌های دیتاست 3 و 4 در مدل بعد از unlearn.

In [ ]:
privacy_losses_train_1 = compute_losses(model11, dataset)
privacy_losses_test_1 = compute_losses(model11, test_subset1)
privacy_losses_train_2 = compute_losses(model11, other_dataset)
privacy_losses_test_2 = compute_losses(model11, test_subset2)

# Combine and cross-validate for privacy model
clf_privacy_1 = train_attacker_model(privacy_losses_train_1, privacy_losses_test_1)
clf_privacy_2 = train_attacker_model(privacy_losses_train_2, privacy_losses_test_2)

X_privacy_1 = privacy_losses_train_1 + privacy_losses_test_1
y_privacy_1 = [1] * len(privacy_losses_train_1) + [0] * len(privacy_losses_test_1)
cv_score_privacy_1 = cross_val_score(clf_privacy_1, X_privacy_1, y_privacy_1)

X_privacy_2 = privacy_losses_train_2 + privacy_losses_test_2
y_privacy_2 = [1] * len(privacy_losses_train_2) + [0] * len(privacy_losses_test_2)
cv_score_privacy_2 = cross_val_score(clf_privacy_2, X_privacy_2, y_privacy_2)

# Print the cross-validation scores
print("Cross-validation score for privacy model (subset 1):", cv_score_privacy_1)
print("Cross-validation score for privacy model (subset 2):", cv_score_privacy_2)

Cross-validation score for privacy model (subset 1): 0.9336915646023805
Cross-validation score for privacy model (subset 2): 0.907563025210084


Part 1.4

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split, Subset
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
import numpy as np
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the dataset and transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Function to create shards and slices
def create_shards_and_slices(dataset, S, R):
    shards = random_split(dataset, [len(dataset) // S] * S)
    slices = [random_split(shard, [len(shard) // R] * R) for shard in shards]
    return slices

# Function to train model on slices
def train_on_slices(model, slices, epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)
    for i, slice_set in enumerate(slices):
        slice_dataset = ConcatDataset(slice_set)
        dataloader = DataLoader(slice_dataset, batch_size=32, shuffle=True)
        for epoch in range(epochs[i]):
            model.train()
            for images, labels in dataloader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        torch.save(model.state_dict(), f'model_slice_{i}.pth')
    return model

# Define evaluation metrics
def evaluate_model(model, test_loader):
    model.eval()
    y_true, y_pred = [], []
    y_score = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            y_score.extend(outputs.cpu().numpy())
    f1 = f1_score(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    y_true_one_hot = torch.nn.functional.one_hot(torch.tensor(y_true), num_classes=10).numpy()
    auroc = roc_auc_score(y_true_one_hot, y_score, multi_class='ovr')
    return f1, accuracy, precision, recall, auroc

# Function to poison the dataset
def poison_data(dataset, class_to_poison, num_samples, block_size=(3, 3)):
    indices = [i for i, (_, label) in enumerate(dataset) if label == class_to_poison]
    selected_indices = random.sample(indices, num_samples)
    poisoned_data = []
    for idx in range(len(dataset)):
        image, label = dataset[idx]
        if idx in selected_indices:
            image = image.numpy()
            c, h, w = image.shape
            x = random.randint(0, w - block_size[1])
            y = random.randint(0, h - block_size[0])
            image[:, y:y + block_size[0], x:x + block_size[1]] = 0
            image = torch.tensor(image)
        poisoned_data.append((image, label))
    return poisoned_data, selected_indices

# Function to unlearn data
def unlearn_data(model, data_to_forget, epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)
    dataloader = DataLoader(data_to_forget, batch_size=32, shuffle=True)
    for epoch in range(epochs):
        model.train()
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    return model

# Load dataset and create DataLoader
class_to_poison = 3  # Example class to poison
poisoned_data, poisoned_indices = poison_data(dataset, class_to_poison, num_samples=500)
poisoned_dataset = Subset(poisoned_data, range(len(poisoned_data)))

slices = create_shards_and_slices(poisoned_dataset, S=5, R=5)
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 10)  # Adjusting for CIFAR-10
model = train_on_slices(model, slices, epochs=[10, 10, 10, 10, 10])
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
f1, accuracy, precision, recall, auroc = evaluate_model(model, test_loader)
print(f'Poisoned Model - F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

# Calculate ASR
def calculate_asr(model, test_loader, target_class):
    model.eval()
    total_samples = 0
    misclassified_as_target = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            misclassified_as_target += (predicted == target_class).sum().item()
    asr = misclassified_as_target / total_samples * 100
    return asr

asr = calculate_asr(model, test_loader, target_class=class_to_poison)
print(f'Attack Success Rate (ASR): {asr}%')

# Unlearn the poisoned data and evaluate again
data_to_forget = Subset(poisoned_dataset, poisoned_indices)
model = unlearn_data(model, data_to_forget, epochs=10)
f1, accuracy, precision, recall, auroc = evaluate_model(model, test_loader)
print(f'Unlearned Model - F1 Score: {f1}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

# Calculate ASR after unlearning
asr = calculate_asr(model, test_loader, target_class=class_to_poison)
print(f'Attack Success Rate (ASR) after unlearning: {asr}%')

100%|██████████| 170498071/170498071 [00:01<00:00, 99923154.77it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 95.7MB/s]


NameError: name 'ConcatDataset' is not defined